In [20]:
import json
import random
from random import choice
from tkinter import _flatten
from tqdm import tqdm

In [21]:
allentity = []

In [22]:
# get all entity
def getcandidate(filepath):
    entity = []
    with open(filepath) as f:
        diclist = json.load(f)['entries']
        for dic in diclist:
            dic = list(dic.values())[0]
            for ops in dic["modifiedtripleset"]:
                entity.append(' '.join(ops['subject'].split('_')).replace('"', '').lower())
                entity.append(' '.join(ops['object'].split('_')).replace('"', '').lower())
    return entity

In [23]:
# change entity
def change(wd):
    if random.random() < 0.5:
        return choice(allentity), True
    else:
        return wd, False

In [24]:
# change sentence and mark where changed
def makenoisy(gold, datadiclist, datastr):
    noisytext = [gold]
    # sum = 0
    entitys = []
    # get all entity in sentence 
    for datadic in datadiclist:
        entitys.append(' '.join(datadic['subject'].lower().split('_')))
        entitys.append(' '.join(datadic['object'].lower().split('_')))
    # split by entity
    for entity in entitys:
        noisytext = [x.split(entity) for x in noisytext]
        z = []
        for x in noisytext:
            tmp = [x[0].strip()]
            for y in x[1:]:
                tmp.append(entity)
                tmp.append(y.strip())
            z.append(tmp)
        noisytext = [x for x in list(_flatten(z)) if x]

    changedid = []
    originid = []
    retext = []

    # mark where changed
    # wd is not a word but words
    for i, wd in enumerate(noisytext):
        if wd in entitys:
            nwd, wh = change(wd)
            if wh:
                changedid.append(i)
                originid.append(entitys.index(wd))
                wd = nwd
        retext.append(wd)
    
    prelen = 0
    errortextpos = ''
    corredatapos = ''

    # from index to position
    for i, wd in enumerate(retext):
        if i in changedid:
            errortextpos+='\t'+str(prelen)+' '+str(prelen+len(wd.split(' ')))
        prelen += len(wd.split(' '))
    # data position
    for i in originid:
        lis = datastr.split(entitys[i])
        corredatapos+='\t'+str(len(lis[0].strip().split(' ')))+' ' + str(len(lis[0].strip().split(' '))+len(entitys[i].split(' ')))
    return ' '.join(retext), errortextpos[1:], corredatapos[1:]

In [25]:
# change file, just some form
def makepseudo(filepath):
    with open(filepath) as f:
        fsrcdata = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.src.data', 'w')
        fsrctext = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.src.text', 'w')
        ftgttext = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.text', 'w')
        ftgtdatapos = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.datapos', 'w')
        ftgttextpos = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.textpos', 'w')

        diclist = json.load(f)['entries']
        for dic in tqdm(diclist):
            dic = list(dic.values())[0]
            dic.pop('originaltriplesets')
            textlist = dic.pop('lexicalisations')
            textlist = [x['lex'] for x in textlist]
            datastr = '<category> '+dic['category']+' <type> '+dic['shape_type']+' <sep>'
            for datadic in dic["modifiedtripleset"]:
                datastr += ' <s> '+' '.join(datadic['subject'].split('_')).lower()+' <p> '+datadic['property']+' <o> '+' '.join(datadic['object'].split('_')).lower()
            for text in textlist:
                text = text.lower()
                ftgttext.write(text+'\n')
                noisytext, errortextpos, corredatapos = makenoisy(text, dic["modifiedtripleset"], datastr)
                fsrctext.write(noisytext + '\n')
                fsrcdata.write(datastr + '\n')
                ftgtdatapos.write(corredatapos + '\n')
                ftgttextpos.write(errortextpos + '\n')
        ftgttext.close() 
        fsrcdata.close()
        fsrctext.close()
        ftgttextpos.close()
        ftgtdatapos.close()

In [26]:
allentity=getcandidate('webnlg_release_v2.1_dev.json') + getcandidate('webnlg_release_v2.1_test.json') + getcandidate('webnlg_release_v2.1_train.json')


In [27]:
makepseudo('webnlg_release_v2.1_test.json')
makepseudo('webnlg_release_v2.1_dev.json')
makepseudo('webnlg_release_v2.1_train.json')

100%|██████████| 12876/12876 [00:01<00:00, 9717.49it/s]


In [28]:
len(allentity)

93588